In [ ]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path

os.chdir(Path(os.getcwd()).parent)
os.getcwd()

In [ ]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
from typing import List, Tuple

Constants

In [ ]:
LANES_MODEL_RESULTS_PATH = '/home/tamir/workspace/AB_AutoTagging_Lanes/tmp/sf_rbs_model_results'
PREDICTION_PATH='/home/tamir/workspace/AB_AutoTagging_Lanes/tmp/b2b_pred_with_rbs.tsv'
DATASET_PATH='/home/tamir/datasets/IL_weekend_run/64eef8f8ac243656c2bd2b76/'

Load autolabel predictions

In [ ]:
pred_df=pd.read_csv(PREDICTION_PATH,sep='\t')
pred_df.head()

In [ ]:
from ab_autotagging_lanes.single_frame.adapter import LaneDetectionModel

df_lanes, df_rbs = LaneDetectionModel.read_and_parse_results(results_path=LANES_MODEL_RESULTS_PATH)
df_rbs.head()

In [ ]:
from ab_autotagging_lanes.road_boundaries_filter import RoadBoundaryFilter, RoadBoundaryFilterParams

rbs_filter = RoadBoundaryFilter(params=RoadBoundaryFilterParams(images_path=DATASET_PATH))


image_name = '1693150350999951.png'
df_rbs_single=df_rbs[df_rbs.name == image_name]
detections = pred_df[pred_df.name==image_name]

rbs_filter.apply_filter_SF(detections=detections, df_rbs_single=df_rbs_single, plot=True, image_name=image_name)

In [ ]:
for image_name in np.random.choice(df_rbs.name.unique(), 50, replace=False):
    print(image_name)
    df_rbs_single=df_rbs[df_rbs.name == image_name]
    detections = pred_df[pred_df.name==image_name]

    rbs_filter.apply_filter_SF(detections=detections, df_rbs_single=df_rbs_single, plot=True, image_name=image_name)

### Special Cases

In [ ]:
special_cases = [
    '1693154053000216.png',
    '1693150350999951.png',
    '1693150393000194.png',
    '1693153105999965.png',
    '1693153187000162.png',
    '1693150238999860.png',
    '1693152045000051.png',
    '1693154748000235.png',
]
# special_cases = [
#     '1693154342000195.png',
#     '1693151118999896.png'
# ]

for image_name in special_cases:
    df_rbs_single=df_rbs[df_rbs.name == image_name]
    detections = pred_df[pred_df.name==image_name]

    rbs_filter.apply_filter(detections=detections, df_rbs_single=df_rbs_single, plot=True, image_name=image_name)

## Statistics

#### Total rbs groups per image

In [ ]:
plt.title(f'Total rbs groups per image')
plt.hist(df_rbs.groupby('name').anchor_idx.nunique().values)
plt.show()

#### Size of rbs anchor groups

In [ ]:
size_of_group_ths = 50

plt.title(f'Size of anchor groups, ths={size_of_group_ths}')
plt.hist(df_rbs.groupby(['name', 'anchor_idx']).size(), bins=50)
plt.axvline(x=size_of_group_ths, color='red')
plt.show()

## Apply On Detections DataFrame

In [ ]:
b2b_pred_path = '/home/tamir/workspace/AB_AutoTagging_Lanes/tmp/b2b_pred_with_rbs.tsv'
b2b_pred_df = pd.read_csv(b2b_pred_path, sep='\t')
b2b_pred_df['is_inside_boundaries'] = None

prod_pred_path = '/home/tamir/workspace/AB_AutoTagging_Lanes/tmp/prod_pred_with_rbs.tsv'
prod_pred_df = pd.read_csv(prod_pred_path, sep='\t')
prod_pred_df['is_inside_boundaries'] = None

In [ ]:
from tqdm import tqdm

for image_name in tqdm(df_rbs.name.unique()):
    df_rbs_single=df_rbs[df_rbs.name == image_name]
    
    if len(df_rbs_single) == 0:
        continue
    
    b2b_detections = b2b_pred_df[b2b_pred_df.name==image_name]
    if len(b2b_detections) > 0:
        is_inside_road_boundary, fit_meta = rbs_filter.apply_filter(detections=b2b_detections, df_rbs_single=df_rbs_single, plot=False, image_name=image_name)
        b2b_pred_df.loc[b2b_pred_df.name == image_name, 'is_inside_boundaries'] = is_inside_road_boundary
        
    prod_detections = prod_pred_df[prod_pred_df.name == int(image_name[:-4])]
    if len(prod_detections) > 0:
        is_inside_road_boundary, fit_meta = rbs_filter.apply_filter(detections=prod_detections, df_rbs_single=df_rbs_single, plot=False, image_name=image_name)
        prod_pred_df.loc[prod_pred_df.name == int(image_name[:-4]), 'is_inside_boundaries'] = is_inside_road_boundary
    
print('Done !')

In [ ]:
b2b_pred_df.to_csv('/home/tamir/workspace/AB_AutoTagging_Lanes/tmp/b2b_pred_with_rbs_v2.tsv', sep="\t") 
prod_pred_df.to_csv('/home/tamir/workspace/AB_AutoTagging_Lanes/tmp/prod_pred_with_rbs_v2.tsv', sep="\t") 
